# Sentiment Model Quickstart (Minimal)

本笔记演示如何在干净环境中快速准备依赖、加载情感模型，并对单条新闻进行推理。


## 1. 准备 Conda 环境

```shell
conda create -n finance python=3.10 -y
conda activate finance
pip install -r requirements.txt
# 若 requirements.txt 缺失依赖，可补装：
# pip install torch transformers peft pandas
```

> 以上命令请在项目根目录的终端中执行（例如 Anaconda Prompt）。


## 2. 准备模型与数据文件

- 将基础模型（如 `Qwen`）放在 `Finance/Qwen` 或任意可访问位置。
- 将微调后的情感模型权重放在 `Finance/qwen_sentiment_model`。
- 确保测试数据 `nasdaq_news_sentiment/1.csv` 已生成（可运行同目录下的 `1.ipynb`）。
- 如路径不同，请在下一单元里的变量手动修改。


In [ ]:
from pathlib import Path

from test_qwen_sentiment import load_trained_sentiment_model, predict_sentiment

# 根据实际路径调整
MODEL_DIR = Path("qwen_sentiment_model").resolve()
BASE_MODEL_DIR = Path("Qwen").resolve()

model, tokenizer = load_trained_sentiment_model(model_path=str(MODEL_DIR))

sample_news = "Apple announces new AI-powered MacBook lineup"
predicted_score = predict_sentiment(model, tokenizer, sample_news, stock_symbol="AAPL")

print(f"新闻: {sample_news}")
print(f"情感评分: {predicted_score}")


In [ ]:
import pandas as pd

csv_path = Path("nasdaq_news_sentiment") / "1.csv"

df = pd.read_csv(csv_path)
df = df[df["Lsa_summary"].notna() & df["sentiment_deepseek"].notna()].head(5)

for _, row in df.iterrows():
    text = row["Lsa_summary"]
    stock_symbol = row.get("Stock_symbol", "STOCK")
    predicted = predict_sentiment(model, tokenizer, text, stock_symbol=stock_symbol)
    print("-" * 40)
    print(f"股票: {stock_symbol}")
    print(f"新闻: {text[:120]}...")
    print(f"真实: {row['sentiment_deepseek']}   预测: {predicted}")


## 3. 下一步

- 若需在 CPU 上运行，可在 `test_qwen_sentiment.py` 中将 `torch_dtype` 改为 `torch.float32`、`device_map` 改为 `"cpu"`。
- 想集成到 Agent，只需在业务代码中调用 `predict_sentiment` 并传入新闻摘要。
- 运行完毕后，`conda deactivate` 退出虚拟环境。
